# Загрузка библиотек

In [1]:
pip install yfinance

  Preparing metadata (setup.py) ... done
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=e63ff6f11d329db7316dab7a0bb537a39a000362583c5a5df06e045d63670133
  Stored in directory: /root/.cache/pip/wheels/34/ba/79/c0260c6f1a03f420ec7673eff9981778f293b9107974679e36
Successfully built multitasking
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install git+https://github.com/prog815/progOrder

  Cloning https://github.com/prog815/progOrder to /tmp/pip-req-build-soqrxwwl
  Running command git clone --filter=blob:none --quiet https://github.com/prog815/progOrder /tmp/pip-req-build-soqrxwwl
  Resolved https://github.com/prog815/progOrder to commit da7c7d38cd9609ee97bcf13032c088475c8a259b
  Preparing metadata (setup.py) ... done
  Created wheel for progOrder: filename=progOrder-0.0.5-py3-none-any.whl size=2628 sha256=b2799354a83727776babc1ad86e10264f87e90c6eba0ded9ede26af9ff04db99
  Stored in directory: /tmp/pip-ephem-wheel-cache-uic9mluj/wheels/90/27/db/48a874d26924c492944ffd71d357e9002e1e70cc4e04dede4f
Successfully built progOrder
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import progOrder

# Загрузка котировок

In [4]:
data = yf.download('SBER.ME',period='10y')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-04-26,94.080002,94.279999,91.610001,92.699997,51.417816,99341000
2012-04-27,92.300003,94.120003,91.849998,93.779999,52.016861,116572050
2012-05-02,94.470001,95.690002,93.059998,93.120003,51.650784,106143030
2012-05-03,93.300003,93.639999,92.120003,92.889999,51.523201,10646420
2012-05-04,92.120003,92.639999,90.320000,90.900002,50.419411,154978600
...,...,...,...,...,...,...
2022-04-20,121.000000,125.440002,117.510002,121.500000,121.500000,66603200
2022-04-21,121.500000,121.500000,121.500000,121.500000,121.500000,0
2022-04-22,118.650002,122.320000,115.550003,116.970001,116.970001,57466590


# Подготовка данных для модели

In [5]:
window = 5 
test_part = 0.2

# подготовка входов
df = pd.DataFrame(index=data.index)
df['dClose'] = data['Close'].pct_change()

# оконный сдвиг
for name in df.columns:
    for n in range(1,window):
        df[name + "_" + str(n)] = df[name].shift(n)

# добавляем дату
df['dow'] = df.index.day_of_week
df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)

df['dom'] = data.reset_index()['Date'].dt.day.values
df['dom_sin'] = np.sin(2 * np.pi * df['dom'] / 31)
df['dom_cos'] = np.cos(2 * np.pi * df['dom'] / 31)

df['month'] = data.reset_index()['Date'].dt.month.values
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

df['week'] = data.reset_index()['Date'].dt.isocalendar().week.values - 1
df['week_sin'] = np.sin(2 * np.pi * df['month'] / 53)
df['week_cos'] = np.cos(2 * np.pi * df['month'] / 53)

# размножаем на два типа ордеров
data_buy = df.copy()
data_sell = df.copy()

# добавляем таргет
ohlc = data.values[:,:4]
buy,sell = progOrder.getBuySellForData(ohlc)
data_buy['Target'] = buy
data_sell['Target'] = sell

# выделяем размер тестовой

test_len = int((data.shape[0]-window-1)*test_part)

# выделяем обучающую

data_buy_train = data_buy.iloc[window+1:-test_len-1,:]
data_sell_train = data_sell.iloc[window+1:-test_len-1,:]

# выделяем тестовоую

data_buy_test = data_buy.iloc[-test_len-1:-1,:]
data_sell_test = data_sell.iloc[-test_len-1:-1,:]

# выделяем расчетную

data_buy_calc = data_buy.iloc[[-1],:]
data_sell_calc = data_sell.iloc[[-1],:]

data_buy_test

,dClose,dClose_1,dClose_2,dClose_3,dClose_4,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos,week,week_sin,week_cos,Target
Date,,,,,,,,,,,,,,,,,,
2020-04-10,-0.007956,0.016576,0.013459,0.036340,0.027257,4,-0.433884,-0.900969,10,0.897805,-0.440394,4,0.866025,-0.5,14,0.456629,0.889657,0.0
2020-04-13,-0.018466,-0.007956,0.016576,0.013459,0.036340,0,0.000000,1.000000,13,0.485302,-0.874347,4,0.866025,-0.5,15,0.456629,0.889657,0.0
2020-04-14,-0.004489,-0.018466,-0.007956,0.016576,0.013459,1,0.781831,0.623490,14,0.299363,-0.954139,4,0.866025,-0.5,15,0.456629,0.889657,0.0
2020-04-15,-0.057253,-0.004489,-0.018466,-0.007956,0.016576,2,0.974928,-0.222521,15,0.101168,-0.994869,4,0.866025,-0.5,15,0.456629,0.889657,0.0
2020-04-16,0.013060,-0.057253,-0.004489,-0.018466,-0.007956,3,0.433884,-0.900969,16,-0.101168,-0.994869,4,0.866025,-0.5,15,0.456629,0.889657,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19,-0.028664,-0.053713,0.014181,-0.047601,0.000886,1,0.781831,0.623490,19,-0.651372,-0.758758,4,0.866025,-0.5,15,0.456629,0.889657,0.0
2022-04-20,0.009975,-0.028664,-0.053713,0.014181,-0.047601,2,0.974928,-0.222521,20,-0.790776,-0.612106,4,0.866025,-0.5,15,0.456629,0.889657,0.0
2022-04-21,0.000000,0.009975,-0.028664,-0.053713,0.014181,3,0.433884,-0.900969,21,-0.897805,-0.440394,4,0.866025,-0.5,15,0.456629,0.889657,0.0


In [6]:
# контроль средних
pd.DataFrame([data_buy_train.describe().loc['mean',:],data_buy_test.describe().loc['mean',:]],index=['train','test'])

,dClose,dClose_1,dClose_2,dClose_3,dClose_4,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos,week,week_sin,week_cos,Target
train,0.000622,0.000599,0.000582,0.000562,0.000545,2.014099,0.353697,-0.088386,16.028197,-0.017754,-0.031864,6.613797,-0.027931,-0.002998,26.113797,0.649739,0.650681,0.022659
test,-0.000525,-0.000491,-0.000389,-0.000316,-0.000281,1.987903,0.352705,-0.073237,15.887097,-0.012486,-0.040624,6.675403,-0.034617,-0.018343,26.296371,0.655383,0.646658,0.030242


In [7]:
# контроль стандартного отклонения
pd.DataFrame([data_buy_train.describe().loc['std',:],data_buy_test.describe().loc['std',:]],index=['train','test'])

,dClose,dClose_1,dClose_2,dClose_3,dClose_4,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos,week,week_sin,week_cos,Target
train,0.020013,0.020020,0.020024,0.020008,0.020000,1.409504,0.514679,0.776291,8.701267,0.713582,0.699981,3.426874,0.709933,0.704066,14.910181,0.281005,0.274894,0.148850
test,0.031148,0.031158,0.031120,0.031164,0.031185,1.415590,0.512999,0.780273,8.682000,0.711579,0.702756,3.404910,0.700513,0.713977,14.961326,0.277894,0.274572,0.171425


# Создание модели

In [8]:
pip install -U lightautoml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.2/296.2 KB 958.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 20.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.0/425.0 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 26.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 692.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 17.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 18.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 18.7 MB/s eta 0:00:0000:0100:01

In [9]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [13]:
task = Task('binary')
roles = { 'target': 'Target'}
automl = TabularAutoML(task)

# Обучение модели

In [14]:
# oof_pred = automl.fit_predict(data_buy_train,roles=roles)

In [21]:
# (oof_pred.data > 0.5).mean()

0.0

# Тестирование модели